# เครือข่ายการสร้างสรรค์

Recurrent Neural Networks (RNNs) และรูปแบบเซลล์ที่มีการควบคุม เช่น Long Short Term Memory Cells (LSTMs) และ Gated Recurrent Units (GRUs) ได้มอบกลไกสำหรับการสร้างแบบจำลองภาษา กล่าวคือ พวกมันสามารถเรียนรู้การเรียงลำดับคำและให้การคาดการณ์คำถัดไปในลำดับได้ สิ่งนี้ทำให้เราสามารถใช้ RNNs สำหรับ **งานสร้างสรรค์** เช่น การสร้างข้อความทั่วไป การแปลภาษา และแม้กระทั่งการสร้างคำบรรยายภาพ

ในสถาปัตยกรรม RNN ที่เราได้พูดถึงในหน่วยก่อนหน้า แต่ละหน่วย RNN จะสร้างสถานะซ่อนถัดไปเป็นผลลัพธ์ อย่างไรก็ตาม เราสามารถเพิ่มผลลัพธ์อีกตัวให้กับแต่ละหน่วย RNN ซึ่งจะช่วยให้เราสามารถสร้าง **ลำดับ** (ที่มีความยาวเท่ากับลำดับต้นฉบับ) นอกจากนี้ เรายังสามารถใช้หน่วย RNN ที่ไม่รับข้อมูลเข้าในแต่ละขั้นตอน โดยใช้เพียงเวกเตอร์สถานะเริ่มต้น และสร้างลำดับของผลลัพธ์ออกมา

ในโน้ตบุ๊กนี้ เราจะมุ่งเน้นไปที่โมเดลการสร้างสรรค์แบบง่ายที่ช่วยให้เราสร้างข้อความได้ เพื่อความเรียบง่าย เรามาสร้าง **เครือข่ายระดับตัวอักษร** ซึ่งสร้างข้อความทีละตัวอักษร ในระหว่างการฝึก เราจำเป็นต้องนำข้อความจากคลังข้อมูล และแบ่งมันออกเป็นลำดับตัวอักษร


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## สร้างคลังคำศัพท์ตัวอักษร

ในการสร้างเครือข่ายแบบสร้างสรรค์ระดับตัวอักษร เราจำเป็นต้องแยกข้อความออกเป็นตัวอักษรแต่ละตัวแทนที่จะเป็นคำ `TextVectorization` layer ที่เราเคยใช้ก่อนหน้านี้ไม่สามารถทำได้ ดังนั้นเรามีสองตัวเลือก:

* โหลดข้อความด้วยตนเองและทำการแบ่งคำ 'ด้วยมือ' ตามตัวอย่าง [Keras อย่างเป็นทางการนี้](https://keras.io/examples/generative/lstm_character_level_text_generation/)
* ใช้คลาส `Tokenizer` สำหรับการแบ่งคำในระดับตัวอักษร

เราจะเลือกตัวเลือกที่สอง `Tokenizer` ยังสามารถใช้เพื่อแบ่งคำในระดับคำได้ ดังนั้นจึงสามารถเปลี่ยนจากการแบ่งคำระดับตัวอักษรไปเป็นระดับคำได้อย่างง่ายดาย

ในการแบ่งคำระดับตัวอักษร เราจำเป็นต้องส่งพารามิเตอร์ `char_level=True`:


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

เรายังต้องการใช้โทเค็นพิเศษหนึ่งตัวเพื่อระบุ **จุดสิ้นสุดของลำดับ** ซึ่งเราจะเรียกว่า `<eos>` มาลองเพิ่มมันเข้าไปในคลังคำด้วยตนเอง:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## การฝึก RNN เชิงกำเนิดเพื่อสร้างหัวข้อข่าว

วิธีที่เราจะฝึก RNN เพื่อสร้างหัวข้อข่าวมีดังนี้ ในแต่ละขั้นตอน เราจะนำหัวข้อข่าวหนึ่งหัวข้อมาใส่ใน RNN และสำหรับแต่ละตัวอักษรที่ป้อนเข้าไป เราจะให้เครือข่ายสร้างตัวอักษรถัดไป:

![ภาพแสดงตัวอย่างการสร้างคำว่า 'HELLO' ด้วย RNN](../../../../../translated_images/th/rnn-generate.56c54afb52f9781d.webp)

สำหรับตัวอักษรสุดท้ายของลำดับ เราจะให้เครือข่ายสร้างโทเค็น `<eos>` 

ความแตกต่างหลักของ RNN เชิงกำเนิดที่เราใช้ในที่นี้คือ เราจะนำผลลัพธ์จากแต่ละขั้นตอนของ RNN มาใช้ ไม่ใช่แค่จากเซลล์สุดท้ายเท่านั้น ซึ่งสามารถทำได้โดยการกำหนดพารามิเตอร์ `return_sequences` ให้กับเซลล์ RNN

ดังนั้น ในระหว่างการฝึก ข้อมูลที่ป้อนเข้าเครือข่ายจะเป็นลำดับของตัวอักษรที่ถูกเข้ารหัสในความยาวที่กำหนด และผลลัพธ์จะเป็นลำดับที่มีความยาวเท่ากัน แต่ถูกเลื่อนออกไปหนึ่งตำแหน่งและสิ้นสุดด้วย `<eos>` มินิแบตช์จะประกอบด้วยลำดับหลายชุด และเราจำเป็นต้องใช้ **padding** เพื่อจัดแนวลำดับทั้งหมดให้ตรงกัน

เรามาสร้างฟังก์ชันที่จะเปลี่ยนรูปแบบชุดข้อมูลให้เรา เนื่องจากเราต้องการเติม padding ในระดับมินิแบตช์ เราจะทำการจัดชุดข้อมูลเป็นแบตช์โดยเรียกใช้ `.batch()` ก่อน จากนั้นจึงใช้ `map` เพื่อทำการเปลี่ยนรูปแบบ ดังนั้น ฟังก์ชันการเปลี่ยนรูปแบบจะรับมินิแบตช์ทั้งหมดเป็นพารามิเตอร์:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

สิ่งสำคัญบางอย่างที่เราทำในที่นี้:
* เราเริ่มต้นด้วยการดึงข้อความจริงจาก string tensor
* `text_to_sequences` แปลงรายการของสตริงให้เป็นรายการของ integer tensors
* `pad_sequences` เติมเต็ม tensors เหล่านั้นให้มีความยาวสูงสุด
* สุดท้าย เราทำการ one-hot encode ตัวอักษรทั้งหมด รวมถึงการเลื่อนและเพิ่ม `<eos>` เราจะได้เห็นในไม่ช้าว่าทำไมเราถึงต้องการ one-hot-encoded characters

อย่างไรก็ตาม ฟังก์ชันนี้เป็น **Pythonic** ซึ่งหมายความว่ามันไม่สามารถแปลงเป็น Tensorflow computational graph ได้โดยอัตโนมัติ เราจะพบข้อผิดพลาดหากพยายามใช้ฟังก์ชันนี้โดยตรงในฟังก์ชัน `Dataset.map` เราจำเป็นต้องครอบคลุมการเรียก Pythonic นี้โดยใช้ตัวห่อ `py_function`:


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **หมายเหตุ**: การแยกแยะระหว่างฟังก์ชันการแปลงแบบ Pythonic และ Tensorflow อาจดูซับซ้อนเกินไป และคุณอาจสงสัยว่าทำไมเราไม่แปลงชุดข้อมูลโดยใช้ฟังก์ชัน Python มาตรฐานก่อนที่จะส่งไปยัง `fit` แม้ว่าสิ่งนี้จะสามารถทำได้ แต่การใช้ `Dataset.map` มีข้อได้เปรียบอย่างมาก เพราะกระบวนการแปลงข้อมูลจะถูกดำเนินการโดยใช้กราฟการคำนวณของ Tensorflow ซึ่งใช้ประโยชน์จากการคำนวณด้วย GPU และลดความจำเป็นในการส่งข้อมูลระหว่าง CPU/GPU

ตอนนี้เราสามารถสร้างเครือข่าย generator ของเราและเริ่มการฝึกได้ มันสามารถอิงจาก recurrent cell ใด ๆ ที่เราได้พูดถึงในหน่วยก่อนหน้า (simple, LSTM หรือ GRU) ในตัวอย่างของเราจะใช้ LSTM

เนื่องจากเครือข่ายรับตัวอักษรเป็นอินพุต และขนาดของคำศัพท์ค่อนข้างเล็ก เราไม่จำเป็นต้องมี embedding layer อินพุตที่ถูก one-hot-encoded สามารถส่งตรงไปยัง LSTM cell ได้เลย ชั้นเอาต์พุตจะเป็น `Dense` classifier ที่จะแปลงผลลัพธ์ของ LSTM ให้เป็นตัวเลขโทเค็นที่ถูก one-hot-encoded

นอกจากนี้ เนื่องจากเรากำลังจัดการกับลำดับที่มีความยาวแปรผัน เราสามารถใช้ชั้น `Masking` เพื่อสร้างมาสก์ที่จะละเลยส่วนที่เติมเต็มของสตริงได้ แม้ว่าสิ่งนี้จะไม่จำเป็นอย่างเคร่งครัด เพราะเราไม่ได้สนใจอะไรมากนักที่เกินโทเค็น `<eos>` แต่เราจะใช้มันเพื่อให้ได้ประสบการณ์กับชั้นประเภทนี้ `input_shape` จะเป็น `(None, vocab_size)` โดยที่ `None` บ่งบอกถึงลำดับที่มีความยาวแปรผัน และรูปร่างของเอาต์พุตคือ `(None, vocab_size)` เช่นกัน ดังที่คุณสามารถเห็นได้จาก `summary`:


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## การสร้างผลลัพธ์

เมื่อเราได้ฝึกโมเดลเรียบร้อยแล้ว ขั้นตอนต่อไปคือการใช้มันเพื่อสร้างผลลัพธ์ ก่อนอื่นเราต้องหาวิธีถอดรหัสข้อความที่แสดงในรูปแบบลำดับของตัวเลขโทเค็น สำหรับการทำเช่นนี้ เราสามารถใช้ฟังก์ชัน `tokenizer.sequences_to_texts` ได้ อย่างไรก็ตาม ฟังก์ชันนี้ไม่ทำงานได้ดีนักกับการโทเค็นในระดับตัวอักษร ดังนั้นเราจะใช้พจนานุกรมของโทเค็นจากตัว tokenizer (เรียกว่า `word_index`) สร้างแผนที่ย้อนกลับ และเขียนฟังก์ชันถอดรหัสของเราเอง:


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

ตอนนี้เราจะเริ่มการสร้างข้อความ เราจะเริ่มต้นด้วยสตริง `start` และเข้ารหัสมันเป็นลำดับ `inp` จากนั้นในแต่ละขั้นตอน เราจะเรียกเครือข่ายของเราเพื่อคาดเดาตัวอักษรถัดไป

ผลลัพธ์ของเครือข่าย `out` คือเวกเตอร์ที่มีองค์ประกอบ `vocab_size` ซึ่งแสดงความน่าจะเป็นของแต่ละโทเค็น และเราสามารถหาหมายเลขโทเค็นที่น่าจะเป็นไปได้มากที่สุดโดยใช้ `argmax` จากนั้นเราจะเพิ่มตัวอักษรนี้ลงในรายการโทเค็นที่สร้างขึ้น และดำเนินการสร้างต่อไป กระบวนการนี้ในการสร้างตัวอักษรหนึ่งตัวจะถูกทำซ้ำ `size` ครั้งเพื่อสร้างจำนวนตัวอักษรที่ต้องการ และเราจะหยุดก่อนกำหนดเมื่อพบ `eos_token`


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## การสุ่มผลลัพธ์ระหว่างการฝึกอบรม

เนื่องจากเราไม่มีตัวชี้วัดที่มีประโยชน์ เช่น *ความแม่นยำ* วิธีเดียวที่เราจะเห็นได้ว่าโมเดลของเรากำลังพัฒนาขึ้นคือการ **สุ่ม** สตริงที่สร้างขึ้นระหว่างการฝึกอบรม เพื่อทำสิ่งนี้ เราจะใช้ **callbacks** หรือฟังก์ชันที่เราสามารถส่งผ่านไปยังฟังก์ชัน `fit` และจะถูกเรียกใช้งานเป็นระยะระหว่างการฝึกอบรม


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


ตัวอย่างนี้สร้างข้อความที่ค่อนข้างดีอยู่แล้ว แต่ยังสามารถปรับปรุงให้ดียิ่งขึ้นได้ในหลายวิธี:

* **เพิ่มข้อความ** เราใช้เพียงแค่หัวข้อสำหรับงานนี้ แต่คุณอาจต้องการทดลองใช้ข้อความเต็มรูปแบบ โปรดจำไว้ว่า RNNs ไม่ค่อยเก่งในการจัดการกับลำดับที่ยาวมากนัก ดังนั้นจึงสมเหตุสมผลที่จะตัดข้อความออกเป็นประโยคสั้นๆ หรือฝึกอบรมด้วยความยาวลำดับที่กำหนดไว้ล่วงหน้า `num_chars` (เช่น 256) คุณอาจลองเปลี่ยนตัวอย่างข้างต้นให้เป็นสถาปัตยกรรมแบบนี้ โดยใช้ [ตัวอย่าง Keras อย่างเป็นทางการ](https://keras.io/examples/generative/lstm_character_level_text_generation/) เป็นแรงบันดาลใจ

* **LSTM หลายชั้น** การลองใช้ LSTM 2 หรือ 3 ชั้นก็สมเหตุสมผลเช่นกัน ดังที่เราได้กล่าวถึงในหน่วยก่อนหน้า แต่ละชั้นของ LSTM จะดึงรูปแบบบางอย่างจากข้อความออกมา และในกรณีของตัวสร้างข้อความระดับตัวอักษร เราสามารถคาดหวังได้ว่าชั้น LSTM ระดับล่างจะรับผิดชอบการดึงพยางค์ และชั้นที่สูงขึ้นจะรับผิดชอบคำและการผสมคำ สิ่งนี้สามารถทำได้ง่ายๆ โดยการส่งพารามิเตอร์จำนวนชั้นไปยังตัวสร้าง LSTM

* คุณอาจต้องการทดลองใช้ **GRU units** และดูว่าแบบใดทำงานได้ดีกว่า รวมถึง **ขนาดของชั้นซ่อนที่แตกต่างกัน** ชั้นซ่อนที่ใหญ่เกินไปอาจทำให้เกิดการ overfitting (เช่น เครือข่ายจะเรียนรู้ข้อความแบบเป๊ะๆ) และขนาดที่เล็กเกินไปอาจไม่สร้างผลลัพธ์ที่ดี


## การสร้างข้อความแบบนุ่มนวลและค่าอุณหภูมิ

ในคำจำกัดความก่อนหน้านี้ของ `generate` เราเลือกตัวอักษรที่มีความน่าจะเป็นสูงสุดเป็นตัวอักษรถัดไปในข้อความที่สร้างขึ้นเสมอ ซึ่งส่งผลให้ข้อความมักจะ "วนซ้ำ" ระหว่างลำดับตัวอักษรเดิมซ้ำไปซ้ำมา เช่นในตัวอย่างนี้:
```
today of the second the company and a second the company ...
```

อย่างไรก็ตาม หากเราดูการแจกแจงความน่าจะเป็นสำหรับตัวอักษรถัดไป อาจพบว่าความแตกต่างระหว่างความน่าจะเป็นสูงสุดสองสามตัวไม่ได้มากนัก เช่น ตัวอักษรหนึ่งอาจมีความน่าจะเป็น 0.2 และอีกตัวหนึ่งมี 0.19 เป็นต้น ตัวอย่างเช่น เมื่อมองหาตัวอักษรถัดไปในลำดับ '*play*' ตัวอักษรถัดไปอาจเป็นช่องว่างหรือ **e** (เช่นในคำว่า *player*) ได้เท่าๆ กัน

สิ่งนี้นำเราไปสู่ข้อสรุปว่าไม่ใช่เรื่อง "ยุติธรรม" เสมอไปที่จะเลือกตัวอักษรที่มีความน่าจะเป็นสูงกว่า เพราะการเลือกตัวอักษรที่มีความน่าจะเป็นรองลงมาอาจยังคงนำไปสู่ข้อความที่มีความหมายได้ การสุ่มเลือกตัวอักษรจากการแจกแจงความน่าจะเป็นที่ได้จากผลลัพธ์ของเครือข่ายจึงเป็นวิธีที่ชาญฉลาดกว่า

การสุ่มนี้สามารถทำได้โดยใช้ฟังก์ชัน `np.multinomial` ซึ่งเป็นการใช้งาน **การแจกแจงแบบมัลติโนเมียล** ฟังก์ชันที่ใช้สร้างข้อความแบบ **นุ่มนวล** นี้ถูกกำหนดไว้ด้านล่าง:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

เราได้แนะนำพารามิเตอร์อีกตัวหนึ่งที่เรียกว่า **temperature** ซึ่งใช้เพื่อบ่งบอกว่าเราควรยึดติดกับความน่าจะเป็นสูงสุดมากแค่ไหน หาก temperature เท่ากับ 1.0 เราจะทำการสุ่มแบบ multinomial อย่างยุติธรรม และเมื่อ temperature เพิ่มขึ้นจนถึงค่าอนันต์ ความน่าจะเป็นทั้งหมดจะเท่ากัน และเราจะสุ่มเลือกตัวอักษรถัดไปแบบสุ่ม ในตัวอย่างด้านล่าง เราสามารถสังเกตได้ว่าข้อความจะไม่มีความหมายเมื่อเราเพิ่มค่า temperature มากเกินไป และข้อความจะคล้ายกับข้อความที่ถูกสร้างขึ้นแบบ "วนซ้ำ" เมื่อค่า temperature เข้าใกล้ 0



---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามให้การแปลมีความถูกต้องมากที่สุด แต่โปรดทราบว่าการแปลโดยอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่ถูกต้อง เอกสารต้นฉบับในภาษาดั้งเดิมควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ ขอแนะนำให้ใช้บริการแปลภาษามืออาชีพ เราจะไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความผิดที่เกิดจากการใช้การแปลนี้
